In [1]:
import torch

In [2]:
from comet_ml import Experiment, Optimizer

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import torch
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [4]:
import datetime


In [5]:
import node

In [6]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [7]:
from tabular_hypernet import HypernetworkPCA, TrainingModes, Hypernetwork
from tabular_hypernet.modules import SimpleNetwork
from tabular_hypernet.training_utils import train_slow_step, train_model, train_carthesian
from tabular_hypernet.interfaces import HypernetworkSklearnInterface, SimpleSklearnInterface
# from ipynb.fs.defs.MNIST_benchmark import test_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [8]:
from sklearn.metrics import classification_report, balanced_accuracy_score

In [9]:
import imblearn

In [10]:
import pyhopper

In [11]:
from loguru import logger

logger.add("log_Lymf.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [12]:
import seaborn as sns

In [13]:
DEVICE="cuda:0"

In [14]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

2

In [15]:
GS_METRIC = "balanced_accuracy"

In [16]:
import openml
from openml.datasets import get_dataset

# Load data

In [17]:
DATA = "mfeat"

In [18]:
if DATA == "mfeat":
    fac = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/mfeat-fac", header=None, delim_whitespace=True)
    fou = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/mfeat-fou", header=None, delim_whitespace=True)
    kar = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/mfeat-kar", header=None, delim_whitespace=True)
    mor = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/mfeat-mor", header=None, delim_whitespace=True)
    pix = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/mfeat-pix", header=None, delim_whitespace=True)
    zer = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mfeat/mfeat-zer", header=None, delim_whitespace=True)
    
    mfeat = pd.concat((fac, fou, kar, mor, pix, zer), axis=1)
    
    X = mfeat.values.astype(float)
    y = np.array([int(np.floor(i/200)) for i in range(2000)])
    y = LabelEncoder().fit_transform(y).astype(int)

print(X.shape, len(np.unique(y)))

(2000, 649) 10


In [19]:
mfeat

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,98,236,531,673,607,647,2,9,3,6,...,1.713529,33.810340,9.858915,1.399891,148.138058,326.239452,9.711070,20.007248,47.032578,539.208457
1,121,193,607,611,585,665,7,9,2,4,...,2.590208,35.400531,70.681899,6.674412,155.135985,377.832675,8.140633,44.536711,46.338954,518.496567
2,115,141,590,605,557,627,12,6,3,3,...,1.476285,19.477230,30.093590,7.858211,150.126419,419.565747,4.530921,26.292170,44.574822,549.912691
3,90,122,627,692,607,642,0,6,4,5,...,1.349613,14.179518,30.564085,7.097728,173.840759,441.350376,3.706023,13.432311,51.739930,574.887814
4,157,167,681,666,587,666,8,6,1,4,...,0.667971,8.705403,30.242473,9.015714,167.021185,332.479997,1.806273,23.689300,50.407770,492.227513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,304,242,927,626,564,674,25,17,0,7,...,2.289602,37.077931,229.878330,6.376995,144.500902,393.876660,8.672527,152.791625,43.559482,501.537544
1996,324,330,919,713,609,693,28,16,8,9,...,2.230959,8.122498,224.796064,7.282318,187.335473,353.453408,1.389084,161.716981,53.922259,482.428339
1997,337,299,852,724,581,660,22,14,10,10,...,2.592315,23.227692,122.971707,6.250161,124.337857,359.585224,4.643621,101.472040,37.013903,430.544358
1998,247,261,866,633,565,615,24,15,2,4,...,3.369291,69.833696,175.418772,3.563793,140.067305,486.688709,16.109266,126.684436,41.154583,571.426195


In [20]:
n_classes = len(np.unique(y if not isinstance(y, tuple) else y_train))
n_classes

10

In [21]:
unique, counts = np.unique(y if not isinstance(y, tuple) else y_train, return_counts=True)

print(dict(zip(unique, counts)))

{0: 200, 1: 200, 2: 200, 3: 200, 4: 200, 5: 200, 6: 200, 7: 200, 8: 200, 9: 200}


In [22]:
n_features = X.shape[1] if not isinstance(X, tuple) else X_train.shape[1]
n_features

649

In [23]:
max_size = int(len(X)*0.7) if not isinstance(X, tuple) else len(X_train)

In [24]:
max_size

1400

In [25]:
GS_DATA_SIZE = max_size
GS_DATA_SIZE

1400

In [26]:
# print('dataset size:', len(X), '|', 'max training size:', max_size)

In [27]:
TEST_RUN = False

# Preprocess

In [28]:
np.random.seed(42)

def prepare_data(X, y, size=None):
    if isinstance(X, tuple) and isinstance(y, tuple):
        X_train, X_test = X
        y_train, y_test = y
    else:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, stratify=y)
    # X_train, y_train = imblearn.over_sampling.RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train, X_test, y_train, y_test = [torch.from_numpy(x) for x in [X_train, X_test, y_train, y_test]]
    
    return X_train, X_test, y_train, y_test

In [29]:
X_train, X_test, y_train, y_test = prepare_data(X, y, size=max_size)

## Benchmark

In [30]:
criterion = torch.nn.CrossEntropyLoss()

In [31]:
def _summarize_results(y_pred, y_score, y_test, labels):
    results = []
    for idx, label in enumerate(labels):
        y_pred_filt = y_pred[y_test==idx]
        y_test_filt = y_test[y_test==idx]
        acc = (y_pred_filt==y_test_filt.numpy()).sum()/len(y_test_filt)*100
        results.append({
            "Class": label,
            "Metric": acc
        })
        
    acc = (y_pred==y_test.numpy()).sum()/len(y_test)*100    
    results.append({
        "Class": "Total",
        "Metric": acc
    })
    
    
    results.append({
        "Class": "balanced_accuracy",
        "Metric": balanced_accuracy_score(y_test, torch.from_numpy(y_pred)).item()*100
    })
    
    try:
        results.append({
            "Class": "F1 score",
            "Metric": f1_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "roc_auc",
            "Metric": roc_auc_score(y_test, torch.from_numpy(y_score[:, 1])).item()*100
        })
        results.append({
            "Class": "Precision",
            "Metric": precision_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "Recall",
            "Metric": recall_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
    except ValueError:
        pass
    return results

def test_model(model_fn, data, train_size, label_encoder=None, iters=10, as_numpy=False):
    if TEST_RUN:
        iters = 1
        
    if label_encoder is not None:
        labels = label_encoder.classes_
    else:
        labels = sorted(pd.unique(data[1][0] if isinstance(data[1], tuple) else data[1]))
    
    results = []

    for i in range(iters):
        print('iter', i+1, 'of', iters)
        X_train, X_test, y_train, y_test = prepare_data(*data, train_size)
        
        model = model_fn()

        if as_numpy:
            model.fit(X_train.numpy(), y_train.numpy());
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        results.extend(_summarize_results(y_pred, y_score, y_test, labels))
        
        del model
        torch.cuda.empty_cache()

    dframe = pd.DataFrame.from_dict(results)
#     sns.violinplot(data=dframe[dframe["Class"]!="Loss"], y="Class", x="Metric", orient='h')
    return dframe

### Param search

In [32]:
def pyhopper_best_params(model_fn, param_grid, data_size=GS_DATA_SIZE, metric=GS_METRIC, time="30m", default_params={}, n_jobs=1):
    if TEST_RUN:
        time = 30
        if 'epochs' in param_grid:
            param_grid["epochs"] = pyhopper.choice([10])
        
    logger.info(f"pyhopper_best_params {model_fn.__name__}")
    
    def objective(params):
        torch.cuda.empty_cache()

        model_results = test_model(
                        model_fn(
                            **default_params,
                            **params
                        ),
                        (X, y),
                        data_size,
                        None, 5)
        with open(f"{DATA}_{model_fn.__name__}_params.txt", "a") as f:
            f.write(str(datetime.datetime.now()) + ", " + str(params) + ", " + str(model_results[model_results["Class"]==metric]["Metric"].mean()) + "\n")
        return model_results[model_results["Class"]==metric]["Metric"].mean()

    from pyhopper.callbacks import History
    search = pyhopper.Search(param_grid)

    best_params = search.run(objective, "maximize", time, n_jobs=n_jobs, seeding_ratio=0.5)
    
    with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
            f.write(str(best_params))
    
    print(f"{DATA}_{model_fn.__name__}_{best_params}")
    return best_params, search.history


xgboost_best = 


node_best = 

nn_fn1_best_params = 

nn_fn2_best_params = 

nn_fn3_best_params = 

hp_pca_best_params = 


hp_best_params = 


rf_best = 

# TRAIN MODELS

### XGBoost

In [33]:
from xgboost import XGBClassifier

In [34]:
import xgboost
xgboost.__version__

'1.6.0'

In [35]:
def get_xgboost(**params):
    random_seed = np.random.randint(1024)
    def _inner(**args):
        return XGBClassifier(
            verbosity=0,
            random_state=random_seed,
            use_label_encoder=False,
            **params,
            **args
        )
    return _inner    

#### Hyperparam tuning

In [36]:
param_grid = {
                'n_estimators': pyhopper.choice([50, 100, 250, 500, 1000, 3000]),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbt_best1, xgbt_history1 = pyhopper_best_params(get_xgboost, param_grid, time="90m")

2022-12-21 15:48:15.388 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

In [37]:
param_grid = {
                'subsample': pyhopper.choice([0.5, 0.6, 0.7, 0.8, 0.9, 1]),
                'reg_lambda': pyhopper.float(1e-5, 10, init=0, log=True),
                'reg_alpha': pyhopper.float(1e-5, 10, init=0, log=True),
             }


xgbt_best2, xgbt_history2 = pyhopper_best_params(get_xgboost, param_grid, time="90m", default_params=xgbt_best1)

2022-12-21 17:16:25.156 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

#### Best Params

'Ionosphere' {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 1500}


In [38]:
xgboost_best = {**xgbt_best1, **xgbt_best2}

In [39]:
data_size = max_size

xgb_dframe = test_model(get_xgboost(**xgboost_best),
                        (X, y),
                        data_size,
                        label_encoder=None, iters=10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [40]:
res = xgb_dframe[xgb_dframe["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

1400: 98.03 ~ 0.44 (max: 98.67)


In [41]:
xgb_dframe.groupby(['Class']).mean()

,Metric
Class,
0,99.666667
1,98.333333
2,99.000000
3,96.666667
4,98.333333
5,96.833333
6,98.333333
7,97.000000
8,97.833333


### NODE

In [42]:
from qhoptim.pyt import QHAdam

In [43]:
def node_fn(layer_dim=128, num_layers=1, depth=3, batch_size=32):
    def _inner():
        network = torch.nn.Sequential(
            node.DenseBlock(X_train.shape[1], 
                            layer_dim=layer_dim,
                            num_layers=num_layers, 
                            tree_dim=n_classes+1, 
                            depth=depth, 
                            flatten_output=False,
                            choice_function=node.entmax15, 
                            bin_function=node.entmoid15
                           ),
            node.Lambda(lambda x: x.mean(dim=1))
        )
        
        
        network = network.to(DEVICE)
        network.device=DEVICE
        
        with torch.no_grad():
            res = network(torch.as_tensor(X_train[:1000], device=DEVICE).to(torch.float32))
        
            
        optimizer_params = { 'nus':(0.7, 1.0), 'betas':(0.95, 0.998) }
        optim = QHAdam(network.parameters(), **optimizer_params)
            
        network = SimpleSklearnInterface(network, device=DEVICE, epochs=150, batch_size=batch_size)
        network.optimizer = optim
        return network
    return _inner

#### Tune hyperparams

In [44]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 512, power_of=2), #1024
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
    'batch_size': pyhopper.choice([128]),
}

node_best, node_history = pyhopper_best_params(node_fn, param_grid, time="180m", n_jobs=1)
node_best

2022-12-21 18:49:25.651 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params node_fn


  0%|          | [00:00<?]

Search is scheduled for 03:00:00 (h:m:s)
iter 1 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/qhoptim/pyt/qhadam.py:133: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1_adj).add_(1.0 - beta1_adj, d_p)


iter 2 of 5


...

/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)


iter 5 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)


====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 98.87  : 1     : 03:50 (m:s)     
Random seeding    : 99.1   : 10    : 01:29:04 (h:m:s)
Local sampling    : 98.97  : 22    : 01:23:19 (h:m:s)
Duplicates        : -      : 72    : -               
----------------  : ----   : ----  : ----            
Total             : 99.1   : 105   : 02:56:14 (h:m:s)
mfeat_node_fn_{'layer_dim': 64, 'num_layers': 2, 'depth': 5, 'batch_size': 128}


{'layer_dim': 64, 'num_layers': 2, 'depth': 5, 'batch_size': 128}

#### Use best hyperparams

In [45]:
%%capture --no-stdout

size=max_size

node_results = test_model(node_fn(**node_best),
                    (X, y),
                    size,
                    label_encoder=None, iters=10)
res = node_results[node_results["Class"]=="roc_auc"].reset_index(drop=True)["Metric"]
print(f"{size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10
1400: nan ~ nan, (max: nan)


### Dropout Neural network

In [46]:
def network_fn1(epochs=100, drop1=0.3, drop2=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn2(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn3(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, drop4=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 128),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(128, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop4),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner

#### Find Hyperparams

### Dropout 1 layer

In [47]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2]),
                "batch_size": pyhopper.choice([128]),
             }

nn_fn1_best_params, nn_fn1_history = pyhopper_best_params(network_fn1, param_grid, time="60m")
nn_fn1_best_params

2022-12-21 21:52:20.002 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn1


  0%|          | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 100, 'drop1': 0.5, 'drop2': 0.7, 'lr': 0.003, 'batch_size': 128}

In [48]:
data_size = max_size

nn1_results = test_model(network_fn1(**nn_fn1_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [49]:
res = nn1_results[nn1_results["Class"]=="Balanced Acc score"]["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

1400: nan ~ nan (max: nan)


### Dropout 2 layers

In [50]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2]),
                "batch_size": pyhopper.choice([128]),
             }

nn_fn2_best_params, nn_fn2_history = pyhopper_best_params(network_fn2, param_grid, time="70m")
nn_fn2_best_params

2022-12-21 22:53:25.245 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn2


  0%|          | [00:00<?]

Search is scheduled for 01:10:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 150,
 'drop1': 0.1,
 'drop2': 0.7,
 'drop3': 0.1,
 'lr': 0.0003,
 'batch_size': 128}

In [51]:
data_size = max_size

nn2_results = test_model(network_fn2(**nn_fn2_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### Dropout 3 layers

In [52]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2]),
                "batch_size": pyhopper.choice([128]),
             }

nn_fn3_best_params, nn_fn3_history = pyhopper_best_params(network_fn3, param_grid, time="75m")
nn_fn3_best_params

2022-12-22 00:06:35.152 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn3


  0%|          | [00:00<?]

Search is scheduled for 01:15:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 150,
 'drop1': 0.1,
 'drop2': 0.5,
 'drop3': 0.1,
 'drop4': 0.1,
 'lr': 0.003,
 'batch_size': 128}

In [53]:
data_size = max_size

nn3_results = test_model(network_fn3(**nn_fn3_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### Mask sizes

In [54]:
mask_sizes = [int(n_features * 0.3), int(n_features * 0.5), int(n_features * 0.7), int(n_features * 0.8)]
mask_sizes

[194, 324, 454, 519]

### HypernetworkPCA

In [55]:
def network_pca_fn(epochs=100, masks_no=100, mask_size=100, target_size=100, n_comp=5, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = HypernetworkPCA(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)], 
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_comp, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                        input_size=n_features
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner

In [56]:
def figure_to_html():
    tmpfile = BytesIO()
    plt.gcf().savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

    html = '<img src=\'data:image/png;base64,{}\'>'.format(encoded)
    return html

#### Find hyperparams

In [57]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([5, 7, 20]),
    "mask_size": pyhopper.choice(mask_sizes),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "n_comp": pyhopper.choice([5]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([128]),

}

hp_pca_best_params, hp_pca_history = pyhopper_best_params(network_pca_fn, param_grid, time="480m")
hp_pca_best_params

2022-12-22 01:25:32.402 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_pca_fn


  0%|          | [00:00<?]

Search is scheduled for 08:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
6

iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
i

iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
iter 1 of 5
torch.Size([1, 128])
649
iter 2 of 5
torch.Size([1, 128])
649
iter 3 of 5
torch.Size([1, 128])
649
iter 4 of 5
torch.Size([1, 128])
649
iter 5 of 5
torch.Size([1, 128])
649
i

{'epochs': 100,
 'masks_no': 7,
 'mask_size': 324,
 'target_size': 20,
 'n_comp': 5,
 'lr': 0.0003,
 'batch_size': 128}

In [58]:
hp_pca_best_params

{'epochs': 100,
 'masks_no': 7,
 'mask_size': 324,
 'target_size': 20,
 'n_comp': 5,
 'lr': 0.0003,
 'batch_size': 128}

'Libras'
{'epochs': 150,
 'masks_no': 70,
 'mask_size': 20,
 'target_size': 10,
 'n_comp': 10}

'Lymphography' {'epochs': 120, 'masks_no': 50, 'mask_size': 4, 'target_size': 20, 'n_comp': 6}

In [59]:
DATA

'mfeat'

Ionosphere
{'epochs': 100, 'masks_no': 60, 'mask_size': 5, 'target_size': 10}


In [60]:
epochs = 100#hp_pca_best_params['epochs']
masks_no = hp_pca_best_params['masks_no']
mask_size = hp_pca_best_params['mask_size']
target_size = hp_pca_best_params['target_size']
n_comp = hp_pca_best_params['n_comp']
lr = hp_pca_best_params['lr']
batch_size = hp_pca_best_params['batch_size']
data_size = max_size

nn_pca_results = test_model(network_pca_fn(target_size=target_size, mask_size=mask_size, masks_no=masks_no, n_comp=n_comp, epochs=epochs, verbose=True),
                (X, y),
                data_size,
                None, 10)

# exp.log_table("metrics.csv", nn_pca_results.groupby("Class").mean())
# exp.log_metric("f1_score", nn_pca_results.groupby("Class").mean().loc["F1 score"])

iter 1 of 10
torch.Size([1, 128])
649


100%|██████████| 100/100 [00:56<00:00,  1.76it/s]


iter 2 of 10
torch.Size([1, 128])
649


100%|██████████| 100/100 [00:57<00:00,  1.74it/s]


iter 3 of 10
torch.Size([1, 128])
649


...

100%|██████████| 100/100 [00:57<00:00,  1.74it/s]


iter 9 of 10
torch.Size([1, 128])
649


100%|██████████| 100/100 [00:56<00:00,  1.76it/s]


iter 10 of 10
torch.Size([1, 128])
649


100%|██████████| 100/100 [00:58<00:00,  1.72it/s]


In [61]:
res = nn_pca_results[nn_pca_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

1400: 98.33 ~ 0.51 (max: 99.00)


### Hypernetwork

In [62]:
def network_hp_fn(epochs=150, masks_no=100, mask_size=100, target_size=100, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = Hypernetwork(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)],
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_features, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner


#### Find hyperparams

In [63]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([3, 5, 7, 20]),
    "mask_size": pyhopper.choice(mask_sizes),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([128]),
}

hp_best_params, hp_history = pyhopper_best_params(network_hp_fn, param_grid, time="480m")
hp_best_params

2022-12-22 09:32:21.840 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_hp_fn


  0%|          | [00:00<?]

Search is scheduled for 08:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
it

iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 

iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 

{'epochs': 100,
 'masks_no': 5,
 'mask_size': 519,
 'target_size': 50,
 'lr': 0.0003,
 'batch_size': 128}

In [64]:
hp_best_params

{'epochs': 100,
 'masks_no': 5,
 'mask_size': 519,
 'target_size': 50,
 'lr': 0.0003,
 'batch_size': 128}

#### Train using the best hyperparams

In [65]:
epochs = 100#hp_best_params['epochs']
masks_no = hp_best_params['masks_no']
mask_size = hp_best_params['mask_size']
target_size = hp_best_params['target_size']
data_size = max_size
batch_size = hp_best_params['batch_size']
lr = hp_best_params['lr']


hyper_results = test_model(network_hp_fn(epochs, masks_no, mask_size, target_size, batch_size=batch_size,verbose=True),
                    (X, y),
                    data_size,
                    None, 10)

iter 1 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [00:32<00:00,  3.09it/s]


iter 2 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [00:31<00:00,  3.17it/s]


iter 3 of 10
torch.Size([1, 128])


...

100%|██████████| 100/100 [00:32<00:00,  3.11it/s]


iter 9 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


iter 10 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [00:32<00:00,  3.09it/s]


In [66]:
res = hyper_results[hyper_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

1400: 98.12 ~ 0.81, (max: 99.00)


### Random forest

In [67]:
from sklearn.ensemble import RandomForestClassifier

In [68]:
def get_rf(**params):
    random_seed = np.random.randint(1024)
    def _inner():
        return RandomForestClassifier(
            random_state=random_seed,
            **params
        )
    return _inner

#### Find hyperparams

In [69]:
param_grid = {
                'n_estimators': pyhopper.choice([50, 100, 200, 500, 1000, 3000]),
                'max_features': pyhopper.choice([None, 'sqrt', 0.2, 0.3, 0.5, 0.7]),
                'criterion' : pyhopper.choice(['gini', 'entropy']),
                'max_depth': pyhopper.choice([None, 2, 4, 8, 16]),
             }

rf_best, rf_history = pyhopper_best_params(get_rf, param_grid, time="90m")
rf_best

2022-12-22 17:34:47.365 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_rf


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'n_estimators': 3000,
 'max_features': 'sqrt',
 'criterion': 'entropy',
 'max_depth': 16}

#### Use best params

In [70]:
size = max_size

rf_dframe = test_model(get_rf(**rf_best), 
                        (X, y),
                        size,
                        None, iters=10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


# Collect analytics

In [71]:
d = {}

In [72]:
d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['HypernetPCA'] = nn_pca_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [73]:
all_models_df=pd.concat(d, axis=0)
all_models_df

mean       std         max
              Class                                             
Random forest 0                  98.666667  1.314684  100.000000
              1                  99.666667  0.702728  100.000000
              2                  99.500000  0.805076  100.000000
              3                  98.833333  1.372121  100.000000
              4                  99.666667  0.702728  100.000000
...                                    ...       ...         ...
XGBoost       7                  97.000000  1.892154  100.000000
              8                  97.833333  3.047970  100.000000
              9                  98.333333  1.756821  100.000000
              Total              98.033333  0.436031   98.666667
              balanced_accuracy  98.033333  0.436031   98.666667

[96 rows x 3 columns]

In [74]:
os.environ['COMET_KEY']

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [75]:
all_models_df.to_csv(f"{DATA}_metrics.csv")

In [76]:
exp = Experiment(os.environ.get("COMET_KEY"), 'hypernet-uci-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune2{DATA}")
exp.log_table(f"{DATA}_metrics.csv", all_models_df)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/hypernet-uci-tune/11b6ee48cbdf4c1289742b50841122a5



{'web': 'https://www.comet.com/api/asset/download?assetId=9b883c3a8baa449fb9cde78d73455f89&experimentKey=11b6ee48cbdf4c1289742b50841122a5',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=9b883c3a8baa449fb9cde78d73455f89&experimentKey=11b6ee48cbdf4c1289742b50841122a5',
 'assetId': '9b883c3a8baa449fb9cde78d73455f89'}

### Replace some data in existing

In [77]:
# tmp_df = pd.concat(d, axis=0)
# tmp_df = tmp_df.reset_index()
# tmp_df = tmp_df.rename(columns={tmp_df.columns[0]: DATA})

# tmp_df

In [105]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")
# all_models_df = all_models_df.rename(columns={all_models_df.columns[0]: DATA})
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'Hypernet'].index)
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'HypernetPCA'].index)

# all_models_df

In [106]:
# all_models_df = pd.concat([all_models_df, tmp_df])
# all_models_df

In [77]:
f"{DATA}_metrics.csv"

'mfeat_metrics.csv'

In [78]:
all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

In [79]:
all_models_df.reset_index()

,index,Unnamed: 0,Class,mean,std,max
0,0,Random forest,0,98.666667,1.314684,100.000000
1,1,Random forest,1,99.666667,0.702728,100.000000
2,2,Random forest,2,99.500000,0.805076,100.000000
3,3,Random forest,3,98.833333,1.372121,100.000000
4,4,Random forest,4,99.666667,0.702728,100.000000
...,...,...,...,...,...,...
91,91,XGBoost,7,97.000000,1.892154,100.000000
92,92,XGBoost,8,97.833333,3.047970,100.000000
93,93,XGBoost,9,98.333333,1.756821,100.000000
94,94,XGBoost,Total,98.033333,0.436031,98.666667


In [80]:
# all_models_df = all_models_df.drop(all_models_df.columns[0], axis=1)
# all_models_df

In [81]:
all_models_df[all_models_df['Class'] == 'balanced_accuracy']

,Unnamed: 0,Class,mean,std,max
11,Random forest,balanced_accuracy,98.983333,0.355295,99.500000
23,Hypernet,balanced_accuracy,98.116667,0.805268,99.000000
35,HypernetPCA,balanced_accuracy,98.333333,0.509175,99.000000
47,Dropout_1,balanced_accuracy,98.600000,0.316228,98.833333
59,Dropout_2,balanced_accuracy,98.516667,0.461145,99.333333
71,Dropout_3,balanced_accuracy,98.266667,0.604510,99.000000
83,Node,balanced_accuracy,98.583333,0.446177,99.333333
95,XGBoost,balanced_accuracy,98.033333,0.436031,98.666667


In [82]:
tmp = all_models_df

In [83]:
tmp = tmp.rename(columns={tmp.columns[0]: DATA})

In [84]:
tmp

,mfeat,Class,mean,std,max
0,Random forest,0,98.666667,1.314684,100.000000
1,Random forest,1,99.666667,0.702728,100.000000
2,Random forest,2,99.500000,0.805076,100.000000
3,Random forest,3,98.833333,1.372121,100.000000
4,Random forest,4,99.666667,0.702728,100.000000
...,...,...,...,...,...
91,XGBoost,7,97.000000,1.892154,100.000000
92,XGBoost,8,97.833333,3.047970,100.000000
93,XGBoost,9,98.333333,1.756821,100.000000
94,XGBoost,Total,98.033333,0.436031,98.666667


In [85]:
tmp[tmp['Class'] == "balanced_accuracy"]

,mfeat,Class,mean,std,max
11,Random forest,balanced_accuracy,98.983333,0.355295,99.500000
23,Hypernet,balanced_accuracy,98.116667,0.805268,99.000000
35,HypernetPCA,balanced_accuracy,98.333333,0.509175,99.000000
47,Dropout_1,balanced_accuracy,98.600000,0.316228,98.833333
59,Dropout_2,balanced_accuracy,98.516667,0.461145,99.333333
71,Dropout_3,balanced_accuracy,98.266667,0.604510,99.000000
83,Node,balanced_accuracy,98.583333,0.446177,99.333333
95,XGBoost,balanced_accuracy,98.033333,0.436031,98.666667
